In [1]:
#Importar librerias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#Cargar el dataset
df= pd.read_csv('ncr_ride_bookings.csv')

In [2]:
df.head()

(150000, 21)
Index(['Date', 'Time', 'Booking ID', 'Booking Status', 'Customer ID',
       'Vehicle Type', 'Pickup Location', 'Drop Location', 'Avg VTAT',
       'Avg CTAT', 'Cancelled Rides by Customer',
       'Reason for cancelling by Customer', 'Cancelled Rides by Driver',
       'Driver Cancellation Reason', 'Incomplete Rides',
       'Incomplete Rides Reason', 'Booking Value', 'Ride Distance',
       'Driver Ratings', 'Customer Rating', 'Payment Method'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Date                               150000 non-null  object 
 1   Time                               150000 non-null  object 
 2   Booking ID                         150000 non-null  object 
 3   Booking Status                     150000 non-null  object 
 4   Custome

,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,...,Reason for cancelling by Customer,Cancelled Rides by Driver,Driver Cancellation Reason,Incomplete Rides,Incomplete Rides Reason,Booking Value,Ride Distance,Driver Ratings,Customer Rating,Payment Method
0,2024-03-23,12:29:38,"""CNR5884300""",No Driver Found,"""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-29,18:01:39,"""CNR1326809""",Incomplete,"""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,...,NaN,NaN,NaN,1.0,Vehicle Breakdown,237.0,5.73,NaN,NaN,UPI
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,...,NaN,NaN,NaN,NaN,NaN,627.0,13.58,4.9,4.9,Debit Card
3,2024-10-21,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,...,NaN,NaN,NaN,NaN,NaN,416.0,34.02,4.6,5.0,UPI
4,2024-09-16,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,...,NaN,NaN,NaN,NaN,NaN,737.0,48.21,4.1,4.3,UPI


### Limpieza de valores nulos

Para mantener la calidad de los datos y debido a su contexto, se ha realizado una limpieza condicional de los valores nulos en varias columnas.
Dependiendo de si la columna es númerica o si es de texto se aplicaron las siguientes reglas:

- En las siguientes columnas se han reemplazado los nulos con el valor `No aplica`:
    - `Cancelled Rides by Customer`
    - `Reason for cancelling by Customer`
    - `Cancelled Rides by Driver`
    - `Driver Cancellation Reason`
    - `Incomplete Rides`
    - `Incomplete Rides Reason`
    - `Driver Ratings`
    - `Customer Rating`
    - `Payment Method`
    

- En las siguientes columnas se han reemplazado los nulos con el valor `(-1)`:
    - `Avg VTAT`
    - `Avg CTAT`
    - `Booking Value`
    - `Ride Distance`
    - `Driver Ratings`
    - `Customer Rating`
  
Esta limpieza permite mantener la coherencia en el análisis y evita problemas por algún valor faltante


In [3]:
# Lista de columnas a modificar con "Booking Status" incomplete
columnas_incomplete_texto = [
    'Cancelled Rides by Customer',
    'Reason for cancelling by Customer',
    'Cancelled Rides by Driver',
    'Driver Cancellation Reason'
]
columnas_incomplete_numerica = [
    'Avg CTAT',
    'Avg VTAT',
    'Booking Value',
    'Ride Distance',
    'Driver Ratings',
    'Customer Rating'
]
# Aplicar la limpieza condicional
for columna in columnas_incomplete_texto:
    df.loc[df['Booking Status'] == 'Incomplete', columna] = df.loc[df['Booking Status'] == 'Incomplete', columna].fillna('No aplica')

for columna in columnas_incomplete_numerica:
    df.loc[df['Booking Status'] == 'Incomplete', columna] = df.loc[df['Booking Status'] == 'Incomplete', columna].fillna('-1')
    

# Lista de columnas a modificar con "Booking Status" Completed
columnas_complete_texto = [
    'Cancelled Rides by Customer',
    'Reason for cancelling by Customer',
    'Cancelled Rides by Driver',
    'Driver Cancellation Reason'
]

columnas_complete_numerica = [
    'Incomplete Rides',
    'Incomplete Rides Reason'
]
# Aplicar la limpieza condicional
for columna in columnas_complete_texto:
    df.loc[df['Booking Status'] == 'Completed', columna] = df.loc[df['Booking Status'] == 'Completed', columna].fillna('No aplica')

for columna in columnas_complete_numerica:
    df.loc[df['Booking Status'] == 'Completed', columna] = df.loc[df['Booking Status'] == 'Completed', columna].fillna('-1')


# Lista de columnas a modificar si el viaje fue cancelado por el conductor
columnas_cancelado_por_conductor_texto = [
    'Cancelled Rides by Customer',
    'Reason for cancelling by Customer',
    'Incomplete Rides',
    'Incomplete Rides Reason',
    'Payment Method'
]

columnas_canceladas_por_conductor_numerica = [
    'Avg CTAT',
    'Avg VTAT',
    'Booking Value',
    'Ride Distance',
    'Driver Ratings',
    'Customer Rating'
]

# Aplicar la limpieza condicional
for columna in columnas_canceladas_por_conductor_numerica:
    df.loc[df['Booking Status'] == 'Cancelled by Driver', columna] = df.loc[df['Booking Status'] == 'Cancelled by Driver', columna].fillna('-1')

for columna in columnas_cancelado_por_conductor_texto:
    df.loc[df['Booking Status'] == 'Cancelled by Driver', columna] = df.loc[df['Booking Status'] == 'Cancelled by Driver', columna].fillna('No aplica')


# Lista de columnas a modificar cuando no se encuentra conductor
columnas_no_driver_numerica = [
    'Avg CTAT',
    'Avg VTAT',
    'Booking Value',
    'Ride Distance',
    'Driver Ratings',
    'Customer Rating'
    
]

columnas_no_driver_texto = [
    'Cancelled Rides by Customer',
    'Reason for cancelling by Customer',
    'Cancelled Rides by Driver',
    'Driver Cancellation Reason',
    'Incomplete Rides',
    'Incomplete Rides Reason',
    'Payment Method'
]

# Aplicar la limpieza condicional
for columna in columnas_no_driver_numerica:
    df.loc[df['Booking Status'] == 'No Driver Found', columna] = df.loc[df['Booking Status'] == 'No Driver Found', columna].fillna('-1')

for columna in columnas_no_driver_texto:
    df.loc[df['Booking Status'] == 'No Driver Found', columna] = df.loc[df['Booking Status'] == 'No Driver Found', columna].fillna('No aplica')
    
     # Lista de columnas a modificar cuando la reserva fue cancelada por el cliente
columnas_canceladas_por_cliente_numerica = [
    'Avg CTAT',
    'Avg VTAT',
    'Booking Value',
    'Ride Distance',
    'Driver Ratings',
    'Customer Rating'
]

columnas_canceladas_por_cliente_texto = [
    'Cancelled Rides by Driver',
    'Driver Cancellation Reason',
    'Incomplete Rides',
    'Incomplete Rides Reason',
    'Payment Method'
]

# Aplicar la limpieza condicional
for columna in columnas_canceladas_por_cliente_numerica:
    df.loc[df['Booking Status'] == 'Cancelled by Customer', columna] = df.loc[df['Booking Status'] == 'Cancelled by Customer', columna].fillna('-1')
    
for columna in columnas_canceladas_por_cliente_texto:
    df.loc[df['Booking Status'] == 'Cancelled by Customer', columna] = df.loc[df['Booking Status'] == 'Cancelled by Customer', columna].fillna('No aplica')
    
    



In [4]:
df.isnull().sum()

Date                                 0
Time                                 0
Booking ID                           0
Booking Status                       0
Customer ID                          0
Vehicle Type                         0
Pickup Location                      0
Drop Location                        0
Avg VTAT                             0
Avg CTAT                             0
Cancelled Rides by Customer          0
Reason for cancelling by Customer    0
Cancelled Rides by Driver            0
Driver Cancellation Reason           0
Incomplete Rides                     0
Incomplete Rides Reason              0
Booking Value                        0
Ride Distance                        0
Driver Ratings                       0
Customer Rating                      0
Payment Method                       0
dtype: int64

Nos aseguramos que la columna booking value pase a numerica y convertimos de Rupias a Euros en la columna Booking value para mejor análisis (excepto cuando el valor sea -1)




In [7]:
df['Booking Value'] = pd.to_numeric(df['Booking Value'], errors='coerce')
df['Booking Value'] = df['Booking Value'].apply(lambda x: x if x == -1 else x * 0.0096)

Ahora pasaremos a Power BI para realizar el análisis exploratorio y la visualización ya que es la herramienta elegida para la presentación final

In [13]:
df.to_csv('uber_data_limpio.csv', index=False)